## 本实验依赖与LTP平台，需要先编译好ltp_server并在本机端口12345运行
- 参考Markdowns/Install_ltp.md完成ltp_server的构建

## LTP平台可以实现如下操作，分词（ws）,词性标注（pos），命名实体识别（ner），依存句法分析（dp），语义角色标注（srl）
### 以下是例句 “现在苹果手机在全世界销量正在被华为手机慢慢赶超，两家公司的高管都发表了各自的看法。库克认为华为手机越来越好了。余承东说苹果手机真的贵。”的分析结果
#### 用法参考官方文档
- https://ltp.readthedocs.io/zh_CN/latest/ltpserver.html#id3
- https://ltp.readthedocs.io/zh_CN/latest/appendix.html

In [7]:
import requests as req
import json

In [14]:
# f=json/xml 代表返回格式
# t=分词（ws）,词性标注（pos），命名实体识别（ner），依存句法分析（dp），语义角色标注（srl）或者全部任务（all）
res = req.post(url='http://127.0.0.1:12345/ltp',
data=str('f=json&t=all&s=现在苹果手机在全世界销量正在被华为手机慢慢赶超，两家公司的高管都发表了各自的看法。库克认为华为手机越来越好了。余承东说苹果手机真的贵。').encode('utf-8'))
res_obj = json.loads(res.text)

In [3]:
res_obj[0][0]

[{'arg': [],
  'cont': '现在',
  'id': 0,
  'ne': 'O',
  'parent': 11,
  'pos': 'nt',
  'relate': 'ADV'},
 {'arg': [],
  'cont': '苹果',
  'id': 1,
  'ne': 'O',
  'parent': 2,
  'pos': 'n',
  'relate': 'ATT'},
 {'arg': [],
  'cont': '手机',
  'id': 2,
  'ne': 'O',
  'parent': 11,
  'pos': 'n',
  'relate': 'SBV'},
 {'arg': [],
  'cont': '在',
  'id': 3,
  'ne': 'O',
  'parent': 11,
  'pos': 'p',
  'relate': 'ADV'},
 {'arg': [],
  'cont': '全世界',
  'id': 4,
  'ne': 'O',
  'parent': 3,
  'pos': 'n',
  'relate': 'POB'},
 {'arg': [],
  'cont': '销量',
  'id': 5,
  'ne': 'O',
  'parent': 11,
  'pos': 'n',
  'relate': 'FOB'},
 {'arg': [],
  'cont': '正在',
  'id': 6,
  'ne': 'O',
  'parent': 11,
  'pos': 'd',
  'relate': 'ADV'},
 {'arg': [],
  'cont': '被',
  'id': 7,
  'ne': 'O',
  'parent': 11,
  'pos': 'p',
  'relate': 'ADV'},
 {'arg': [],
  'cont': '华为',
  'id': 8,
  'ne': 'O',
  'parent': 9,
  'pos': 'nz',
  'relate': 'ATT'},
 {'arg': [],
  'cont': '手机',
  'id': 9,
  'ne': 'O',
  'parent': 7,
  'pos'

### 本实验使用了【新闻语料库训练出来的语言模型】和【A*算法】得到与“说”意思相近的词集合
#### 根据常识，新闻中报道人物言论的文字较多，所以新闻语料训练结果会更好，结果也是符合预期，参见*head_words_training.ipynb*
#### 通过wiki_cn语料库（1G左右）训练出来的模型并不理想，故不考虑，参见*word2vec_wiki.ipynb*
*词集合经过简单的人工筛选，去掉了一些明显不合理的词，占比10%左右*

In [39]:
head_words = ['表示','指出','认为','坦言','看来','透露','介绍','明说','说','强调','所说','提到','说道','称','声称','建议','呼吁',
              '提及','地说','直言','普遍认为','批评','重申','提出','明确指出','觉得','宣称','猜测','特别强调','写道','引用','相信',
              '解释','谈到','深知','称赞','感慨','主张','还称','中称','指责','披露','明确提出','描述','提醒','深有体会','爆料',
              '裁定','宣布']

## 对于一篇文章思路是先要找到目标句子

In [8]:
import re

def split_sentences(text):
    text = text.replace('\n','')
    sentences = re.split('(。|！|\!|\.|？|\?)',text) 
 
    new_sents = []
    for i in range(int(len(sentences)/2)):
        sent = sentences[2*i] + sentences[2*i+1]
        new_sents.append(sent)
    return new_sents

In [9]:
import jieba

def get_target_sentences(sentences, keywords):
    target_sentences = []
    for sent in sentences:
        words = jieba.cut(sent)
        if len([word for word in words if word in head_words]) > 0:
            target_sentences.append(sent)
    return target_sentences

In [10]:
with open("news_content.txt",mode='r',encoding='utf-8') as news_file:
    news_content = news_file.read()

In [11]:
target_sentences = get_target_sentences(split_sentences(news_content),head_words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Mark\AppData\Local\Temp\jieba.cache
Loading model cost 1.007 seconds.
Prefix dict has been built succesfully.


In [12]:
target_sentences

['苹果公司回应称，将停售德国商店的iPhone 7以及iPhone 8手机。',
 '苹果在一份声明中表示，计划对这项裁决提出上诉。',
 '法官马提亚·齐根（Matthias Zigann）星期四早些时候对法庭说，在这种情况下，这项裁决不会立即生效。',
 '然而苹果公司表示，在整个上诉过程中，其将不会在位于德国的15家苹果零售商店销售iPhone 7和iPhone 8手机。',
 '苹果在声明中称，最新款的iPhone XS、iPhone XS Max和iPhone XR仍将在这些门店销售。',
 '苹果表示，所有型号的iPhone仍将通过德国的运营商和其他第三方零售商销售。',
 '但高通在一份新闻稿中表示，禁令将在其正式发布后立即生效。',
 '该公司表示，将在“几天内”完成这一过程。',
 '此案是高通第三次因专利侵权指控而申请对苹果iPhone手机实施禁令，此前其在美国和中国也提出过类似申请。',
 '法官裁定，搭载英特尔和苹果供应商Qorvo芯片的手机侵犯了高通一项围绕所谓封包追踪技术 (envelope tracking)的专利。',
 '苹果在一份声明中表示，“高通的行动是孤注一掷的尝试，目的是为了转移人们对两家公司之间真正问题的注意力。',
 '在一份声明中，Qorvo首席知识产权律师迈克·贝克(Mike Baker)说:“我们确信，我们的封包追踪技术芯片在诉讼中没有侵犯专利，如果法庭认真考量了所有证据，就会得出不同的结论。',
 '令我们感到失望的是，我们芯片的发明者和设计师出席了听证会，却没有得到作证或提供其他证据的机会，也无法证明高通的侵权主张是错误的。',
 '”英特尔总法律顾问史蒂芬·罗杰斯(Steven Rodgers)在一篇博客文章中表示，“高通的目标不是维护自己的知识产权，而是排斥高端调制解调器芯片市场的竞争，并捍卫一种最终会损害消费者利益的商业模式。',
 '”高通公司总法律顾问和执行副总裁丹·罗森博格（Don Rosenberg）在一次新闻发布会上表示，“过去两周，不同司法管辖区的两家权威法院已经证实了高通专利的价值，并宣布苹果是侵权者，并在德国和中国这两个重要市场上发布了针对部分iPhone手机的禁售令。',
 '英国《金融时报》周三报道称，苹果曾试图先发制人，通过改变其在德国的软件来防止基于另一项功能的禁售令发布。']

In [15]:
ltp_res = req.post('http://127.0.0.1:12345/ltp', 
data = str('f=json&t=all&s=' + 
'法官马提亚·齐根（Matthias Zigann）星期四早些时候对法庭说，在这种情况下，这项裁决不会立即生效。'
).encode('utf-8'))
ltp_obj = json.loads(ltp_res.text)

## 根据LTP的NER和依存分析结果，如下方法可以找到观点表述人
- 对于一个目标句子（包含目标Head词，即与“说”相近的词）
- 如果目标词的parent为-1，即该词本身就是句子的Head（一句话仅有一个）
- 对于parent为Head的SBV（与Head是主谓关系）词就是Head的主语，即是我们需要的人名（观点表述人）

In [16]:
head_word_obj = [word_obj for word_obj in ltp_obj[0][0] if word_obj['parent'] == -1][0]

In [17]:
SBV_obj = [word_obj for word_obj in ltp_obj[0][0] if word_obj['parent'] == head_word_obj['id'] and word_obj['relate'] == 'SBV'][0]

In [18]:
SBV_obj

{'arg': [],
 'cont': '马提亚·齐根',
 'id': 1,
 'ne': 'S-Nh',
 'parent': 12,
 'pos': 'nh',
 'relate': 'SBV'}

## 一个句子分析出人物言论
+ 获得句子的NER和依赖分析结果
+ 根据依赖分析尝试找到Head，如果Head不属于目标词，舍弃该句子
+ 根据依赖分析尝试找到主语（该主语为人，根据NER），如果找不到，舍弃该句子
+ 保留主语的一个修饰语（ATT，定中关系）

In [45]:
def get_speaker_view_ltp(sentence):
    ltp_res = req.post('http://127.0.0.1:12345/ltp', 
                       data = str('f=json&t=all&s=' + sentence).encode('utf-8'))
    ltp_obj = json.loads(ltp_res.text)
    sent_ltp = ltp_obj[0][0]
    
    # get head and exist if not the target head word
    head_index = -1
    for i in range(len(sent_ltp)):
        if sent_ltp[i]['parent'] == -1 and sent_ltp[i]['cont'] in head_words:
            head_index = i
            break
    if head_index < 0:
        return 0
    head_word_obj = sent_ltp[head_index]    
    
    SBV_index = -1
    for i in range(len(sent_ltp)):
        if sent_ltp[i]['parent'] == head_word_obj['id'] and sent_ltp[i]['relate'] == 'SBV' and sent_ltp[i]['ne'] == 'S-Nh':
            SBV_index = i
            break
    if SBV_index < 0:
        return 0
    
    i = SBV_index - 1
    offset = 0
    while i >= 0 and sent_ltp[i]['relate'] == 'ATT':
        i -= 1
        offset += 1
    speaker = ''.join([word_obj['cont'] for word_obj in sent_ltp[SBV_index - offset: SBV_index + 1]])
    
    if len(sent_ltp) <= head_index: return 0
    
    offset = 0
    if sent_ltp[head_index+1]['relate'] == 'WP':
        offset += 1

    viewpoint = ''.join([word_obj['cont'] for word_obj in sent_ltp[head_index + 1 + offset:]])
    return (speaker, viewpoint)

## 提取新闻人物言论的综合方法
+ 对一篇新闻进行句子拆分
+ 从拆分的句子集合中获得目标句子集合
+ 对于每一个目标句子，尝试找到Head，如果Head不属于目标词，舍弃该句子
+ 对于上一步未舍弃的句子，尝试找到主语（该主语为人），如果找不到，舍弃该句子
+ 保留主语的一个修饰语

In [67]:
def get_news_speaker_and_viewpoints(news_content):
    target_sentences = get_target_sentences(split_sentences(news_content),head_words)
    for sentence in target_sentences:
        speaker_view = get_speaker_view_ltp(sentence)
        if speaker_view and speaker_view[1]:
            print('【' + speaker_view[0] + '】：' + speaker_view[1] + '')

In [68]:
get_news_speaker_and_viewpoints(news_content)

【法官马提亚·齐根】：在这种情况下，这项裁决不会立即生效。
【高通】：禁令将在其正式发布后立即生效。
【Qorvo首席知识产权律师迈克·贝克】：“我们确信，我们的封包追踪技术芯片在诉讼中没有侵犯专利，如果法庭认真考量了所有证据，就会得出不同的结论。
【英特尔总法律顾问史蒂芬·罗杰斯】：“高通的目标不是维护自己的知识产权，而是排斥高端调制解调器芯片市场的竞争，并捍卫一种最终会损害消费者利益的商业模式。


In [69]:
def get_viewpoints(news_file):
    with open(news_file,mode='r',encoding='utf-8') as file:
        content = file.read()
    get_news_speaker_and_viewpoints(content)

In [70]:
get_viewpoints('news/news1.txt')

【孟晚舟】：自己的工作没想到会以这样的形式得到回报：“这份回报以一个普通日本人的来信展现在我面前，让我的心里充满了无比的自豪与宽慰。


In [71]:
get_viewpoints('news/news2.txt')

【德国总理默克尔】：欧盟竞争法在创造全球冠军方面“无法向我们提供足够多的帮助”。


In [72]:
get_viewpoints('news/news3.txt')

【格罗斯】：他的许多同事也持有同样的想法。
【格罗斯】：道，环形正负电子对撞机（CEPC）是项目的第一阶段，可以使正负电子在数千亿电子伏特（几百GeV）的能量下发生碰撞，这也是欧洲核子中心（CERN）发现希格斯玻色子的范围。
【格罗斯】：未来的中国超级对撞机也将如此。
【格罗斯】：中国的超级对撞机进入了新的能量范围，并不会重走欧洲核子中心的路径。
【格罗斯】：这是有史以来最棒的物理学理论，起码有34个诺贝尔奖颁给了与此相关的科学家。
【格罗斯】：理论物理学家总有一种矛盾的心情。
【谢耳朵】：放弃弦理论，转而投奔暗物质门下。


In [73]:
get_viewpoints('news/news4.txt')

【蒋洪义】：目前高通已经在中国市场对苹果发起24件专利侵权诉讼，目前，高通在部分诉讼案件，已经把苹果最新发布的3款iPhone产品追加到诉讼中。
【蒋洪义】：苹果的说法是不成立的，首先福州中院做出的禁令是有效的，在禁令有效或没有被改变的情况下，苹果必须按照禁令不折不扣的执行。
【蒋洪义】：“苹果近期向用户推送软件更新，这与禁令完全没有关系，因为软件更新针对的是消费者已经购买、使用的iPhone产品，这部分手机不属于禁令的目标对象。
【蒋洪义】：根据法律规定，复议不能影响禁令执行。
【蒋洪义】：“苹果如果按照他们所说的尊重中国法律，合规经营，他们不应该做出拒绝接收法律文书的做法。
